# HW01 COVID19

## Import packages

In [9]:
# Numerical Operations
import math

import numpy as np

# Reading/Writing Data
import pandas as pd 
import os
import csv

# For progress Bar
from tqdm import tqdm

# Pytorch
import torch
import torch.nn as nn   
from torch.utils.data import Dataset, DataLoader, random_split

# For plotting learing curve
from torch.utils.tensorboard import SummaryWriter

## Some Utility Functions

In [17]:
def same_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda .is_available():
        torch.cuda.manual_seed_all(seed)

def train_valid_split(data_set, valid_ratio, seed):
    valid_set_size = int(valid_ratio * len(data_set))
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)

def pridict(test_loader, model, device):
    model.eval
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)
        with torch.no_grad():
            pred = model(x)
            preds.append(pred.detach().cpu)
    preds = torch.cat(preds, dim=0).numpy
    return preds

## Dataset

In [18]:
class COVID19Dataset(Dataset):
    def __init(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

def __getitem__(self, idx):
    if self.y is None:
        return self.x[idx]
    else:
        return self.x[idx], self.y[idx]

def __len__(self):
    return len(self.x)

## Neural Network Model

In [21]:
class My_Model(nn.Module):
    def __init__(self, input_dim):
        super(My_Model, self).__init__()

        self.layers = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1)
        return x

## Feature Selection

In [22]:
def select_feat(train_data, valid_data, test_data, select_all=True):
    y_train, y_valid = train_data[:, -1], valid_data[:, -1]
    raw_x_train, raw_x_valid, raw_x_test = train_data[:, :-1], valid_data[:, :-1], test_data

    if select_all:
        feat_idx = list(range(raw_x_train.shape[1]))
    else:
        feat_idx = [0,1,2,3,4,5]
    
    return raw_x_train[:, feat_idx], raw_x_valid[:, feat_idx], raw_x_test[:, feat_idx], y_train, y_valid

## Training Loop

In [23]:
def trainer(train_loader, valid_loader, model, config, device):
    
    criterion = nn.MSELoss(reduction='mean')

    optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum='0.9')

    writer = SummaryWriter()

    if not os.path.isdir('./models'):
        os.mkdir('./models')

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    for epoch in range(n_epochs):
        model.train()
        loss_record = []

        train_pbar = tqdm(train_loader, position=0, leave=True)

        for x, y in train_pbar:
            optimizer.zero_grad()
            x, y = x.to(device), y.to(device)
            pred = model(x)
            loss = criterion(pred, y)
            optimizer.step()
            step += 1
            loss_record.append(loss.detach().item())

            train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            train_pbar.set_posfix({'loss': loss.detach().item()})
            
        mean_train_loss = sum(loss_record)/len(loss_record) 
        writer.add_scalar('Loss/train', mean_train_loss, step)

        model.eval()
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss - criterion(pred, y)

            loss_record.append(loss.item)

        mean_valid_loss = sum(loss_record)/len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path'])
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else:
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return

## Configurations

In [24]:
device = 'cude' if torch.cuda.is_available() else 'cpu'
config = {
    'seed':5201314,
    'select_all': True,
    'valid_ratio': 0.2,
    'n_epochs': 3000,
    'batch_size': 256,
    'learning_rate': 1e-5,
    'early_stop': 400,
    'saveJ_path': './model/model.ckpt'
}

## Dataloader

In [ ]:
# Set seed for reproducibility
same_seed(config['seed'])

# train_data size: 2699 x 118 (id + 37 states + 16 features x 6 days)
# test_data size; 1078 x 117 (without last dat's postive rate)
train_data, test_data = pd.read_csv('.')